# Connect Seatizen SQL database:

Check guidelines with "sf" R package and Postgres / Postgis, eg:

https://rtask.thinkr.fr/fr/interagir-avec-postgis-depuis-r-avec-dbi-et-sf/ 

## Install libraries and connect the database

In [4]:
library(DBI)
library(sf)

# With DBI
con_Reef_database <- dbConnect(
  RPostgres::Postgres(),
  host = "reef-db.d4science.org",
  dbname = "Reef_database",
  port = 5432,
  user = "Reefifi",
  password = "Reefifitoto"
)

## Execute a SQL query with sf "st_read" function

In [ ]:
query <- "select * from view_occurences_manual_annotation where species_name = 'Syringodium isoetifolium' limit 1500;"

Le_morne_sf <- st_read(con_Reef_database, query = query)

Le_morne_sf$st_astext <- st_as_text(Le_morne_sf$geometry_postgis)

colnames(Le_morne_sf)


## Display data in a viewer (leaflet)

In [ ]:
#ensure leaflet is loaded to map the data
library(leaflet)
library(dplyr)
layer = "view_occurences_manual_annotation"
#calculate coordinates of the center of the areas where photos are located
centroid <- st_union(Le_morne_sf$geometry_postgis)   %>% st_convex_hull()   %>% st_centroid()
centroid

    

# set the map with the data and extra layers to make it pretty
leaflet(data = Le_morne_sf,options = leafletOptions(minZoom = 3, maxZoom = 18)
        ) %>% addTiles(
        ) %>% addCircleMarkers(lng =~as_data_frame(st_coordinates(Le_morne_sf$geometry_postgis))$X,
                               lat = ~as_data_frame(st_coordinates(Le_morne_sf$geometry_postgis))$Y,
                               label = ~as.character(id),
                               labelOptions = labelOptions(noHide = F, textsize = "15px"),
                               popup = paste0("<img src=\"data:image/jpeg;base64, ", gsub("base64:","",Le_morne_sf$ThumbnailImage),"\"  style=\"display:block; width:100px;height:100px;\" >")
        ) %>% setView(lng = st_coordinates(centroid)[1], lat =st_coordinates(centroid)[2], zoom = 15
        ) %>% addProviderTiles(providers$Esri.WorldImagery, group = "ESRI World imagery", options = providerTileOptions(opacity = 0.95)
        ) %>% addWMSTiles(
    "https://geoserver-sdi-lab.d4science.org/geoserver/Reef_database/ows",
    layers = layer,
    options = WMSTileOptions(format = "image/png", transparent = TRUE), group ="Seatizen",
      attribution = "Seatizen WMS"
  )  %>% addLayersControl(
    baseGroups = "My Seatizen Map",
    #overlayGroups = c("Seatizen WMS","ESRI World imagery"),
    options = layersControlOptions(collapsed = FALSE)
  ) 
